In [2]:
import pandas as pd
import warnings

In [4]:
# configuração inicial
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=DeprecationWarning)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

In [5]:
# importa os dados do arquivo GeoDados.xlsx
df_ETL = pd.read_excel('GeoDados.xlsx')
df_ETL.head()

,ISO-3166 alpha2,ISO-3166 alpha3,ISO-3166 numeric,fips,País (pt-BR),País (en-US),Capital,Area em km2,População,Continente
0,AD,AND,20,AN,Andorra,Andorra,Andorra la Vella,468.00,77006,EU
1,AE,ARE,784,AE,Emirados Árabes Unidos,United Arab Emirates,Abu Dhabi,82880.00,9630959,AS
2,AF,AFG,4,AF,Afeganistão,Afghanistan,Kabul,647500.00,37172386,AS
3,AG,ATG,28,AC,Antígua e Barbuda,Antigua and Barbuda,St. John's,443.00,96286,NaN
4,AI,AIA,660,AV,Anguila,Anguilla,The Valley,102.00,13254,NaN


In [6]:
df_ETL.columns

Index(['ISO-3166 alpha2', 'ISO-3166 alpha3', 'ISO-3166 numeric', 'fips',
       'País (pt-BR)', 'País (en-US)', 'Capital', 'Area em km2', 'População',
       'Continente'],
      dtype='object')

In [7]:
# contando o número de linhas
df_ETL["País (pt-BR)"].count()

250

In [8]:
# Selecionando as colunas desejadas
colunas_selecionadas = ['ISO-3166 alpha2', 'ISO-3166 alpha3', 'País (pt-BR)', 'Capital', 'Area em km2', 'População']
novo_df = df_ETL[colunas_selecionadas]

novo_df.head()


,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital,Area em km2,População
0,AD,AND,Andorra,Andorra la Vella,468.00,77006
1,AE,ARE,Emirados Árabes Unidos,Abu Dhabi,82880.00,9630959
2,AF,AFG,Afeganistão,Kabul,647500.00,37172386
3,AG,ATG,Antígua e Barbuda,St. John's,443.00,96286
4,AI,AIA,Anguila,The Valley,102.00,13254


In [9]:
# verificando dados faltantes
novo_df.isnull().sum()

ISO-3166 alpha2    1
ISO-3166 alpha3    0
País (pt-BR)       0
Capital            6
Area em km2        0
População          0
dtype: int64

In [10]:
# exibindo os dados faltantes
novo_df[novo_df['Capital'].isnull()]

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital,Area em km2,População
8,AQ,ATA,Antártica,NaN,14000000.00,0
29,BQ,BES,"Bonaire, Santo Eustáquio e Saba",NaN,328.00,18012
33,BV,BVT,Ilha Bouvet,NaN,49.00,0
95,HM,HMD,Ilhas Heard e McDonald,NaN,412.00,0
219,TK,TKL,Toquelau,NaN,10.00,1466
231,UM,UMI,Ilhas Distantes dos EUA,NaN,0.00,0


In [11]:
novo_df[novo_df['ISO-3166 alpha2'].isnull()]

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital,Area em km2,População
159,NaN,NAM,Namíbia,Windhoek,825418.00,2448255


In [12]:
# Preenchendo capitais faltantes com o nome do país
novo_df['Capital'] = novo_df.apply(
    lambda row: row['País (pt-BR)'] if pd.isna(row['Capital']) else row['Capital'], axis=1
)

In [13]:
# exibindo os dados faltantes
novo_df.head()

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital,Area em km2,População
0,AD,AND,Andorra,Andorra la Vella,468.00,77006
1,AE,ARE,Emirados Árabes Unidos,Abu Dhabi,82880.00,9630959
2,AF,AFG,Afeganistão,Kabul,647500.00,37172386
3,AG,ATG,Antígua e Barbuda,St. John's,443.00,96286
4,AI,AIA,Anguila,The Valley,102.00,13254


In [14]:
novo_df[novo_df['Capital'].isnull()]

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital,Area em km2,População


In [15]:
# Corrigindo o valor NaN na coluna 'ISO-3166 alpha2' para o país 'Namíbia'
novo_df.loc[novo_df['País (pt-BR)'] == 'Namíbia', 'ISO-3166 alpha2'] = 'NA'

In [16]:
print(novo_df[novo_df['País (pt-BR)'] == 'Namíbia'])

    ISO-3166 alpha2 ISO-3166 alpha3 País (pt-BR)   Capital  Area em km2  \
159              NA             NAM      Namíbia  Windhoek    825418.00   

     População  
159    2448255  


In [17]:
novo_df[novo_df['ISO-3166 alpha2'].isnull()]

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital,Area em km2,População


In [18]:
novo_df.sample(5)

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital,Area em km2,População
209,SV,SLV,El Salvador,San Salvador,21040.00,6420744
205,SO,SOM,Somália,Mogadishu,637657.00,15008154
51,CV,CPV,Cabo Verde,Praia,4033.00,543767
44,CK,COK,Ilhas Cook,Avarua,240.00,21388
168,NR,NRU,Nauru,Yaren,21.00,12704


In [19]:
# Função para gerar a URL da bandeira
def gerar_url_bandeira(alpha2):
    base_url = 'http://www.geonames.org/flags/x/'
    return f"{base_url}{alpha2.lower()}.gif"

# Adicionando a coluna de URLs das bandeiras
novo_df['URL Bandeira'] = novo_df['ISO-3166 alpha2'].apply(gerar_url_bandeira)

# Exibindo o DataFrame atualizado com as URLs das bandeiras
display(novo_df.head())

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital,Area em km2,População,URL Bandeira
0,AD,AND,Andorra,Andorra la Vella,468.00,77006,http://www.geonames.org/flags/x/ad.gif
1,AE,ARE,Emirados Árabes Unidos,Abu Dhabi,82880.00,9630959,http://www.geonames.org/flags/x/ae.gif
2,AF,AFG,Afeganistão,Kabul,647500.00,37172386,http://www.geonames.org/flags/x/af.gif
3,AG,ATG,Antígua e Barbuda,St. John's,443.00,96286,http://www.geonames.org/flags/x/ag.gif
4,AI,AIA,Anguila,The Valley,102.00,13254,http://www.geonames.org/flags/x/ai.gif


In [20]:
# verificando dados faltantes
novo_df.isnull().sum()

ISO-3166 alpha2    0
ISO-3166 alpha3    0
País (pt-BR)       0
Capital            0
Area em km2        0
População          0
URL Bandeira       0
dtype: int64

In [21]:
# salvando o DataFrame em um arquivo do Excel
novo_df.to_excel('GeoDados_flags.xlsx', index=False)